In [35]:
%pip install -qU langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.


#### Total number of lines in the orignal conversation is around 2200.
#### Batches are made, each containing atmost 200 conversations.
#### Summaries for each batch is created and then combined into a unified summary.

In [120]:
pwd

'/Users/akshitbhatia'

In [97]:
# configuration setup.
import requests
from groq import Groq

key = 'gsk-'
client = Groq(api_key=key)

text_file = Path("Downloads/_chat 6.txt")
conv_text = text_file.open('r').read()

There is no chat log provided. If you'd like to have a conversation and then I can summarize it for you, I'd be happy to chat with you!


In [108]:
# static methods
def add_conversation(content: str, combined: bool=False) -> list:
    """This creates the conversation dictionary.
    
    Parameters
    ----------
    content: string message. if list of messages, combine them with a space and join function.
    combined: boolean flag that when is true, means different summaries needs to be combined.

    Returns
    -------
    returns a list object, containing dictionaries with key being the role and value being the content.
    """
    
    list_convo = []
    if combined:
        list_convo.append({"role":'system','content': system_combined})
    else:
        list_convo.append({"role":'system','content': system_message})
    
    list_convo.append({"role": "user", "content": content})
    
    return list_convo

# make splits of 300
def create_splits(text: str,size=200) -> list:
    """Creates splits such that each contains the given size."""
    splits = []
    texts = text.split("\n")
    print('length of lines in the given conversation is: ',len(texts))
    
    for ind, i in enumerate(range(0,len(texts), size)):
        if i+size >= len(texts):
            splits.append(texts[i:i+len(texts)])
            continue

        splits.append(texts[i:i+size])  

    return splits

In [119]:
class ConvoSummarizer:
    
    system_message='You are a conversation summarizer. For the given part of the conversation, summarize it. Focus on the main themes, key points and topics.'
    system_combined = 'As a person who keeps aggregates important information together, for the list of given summaries, make a unified summary.'

    def __init__(self, client: Groq, model:str = "llama-3.3-70b-versatile", batch_size = None):
        self.model = model
        self.batch_size = batch_size
    
    def create_summaries(self, splits: list) -> list:
        """
        For each split containing text, make a summarization.

        Parameters:
        -----------
        splits: list of strings

        """
        summaries=[]
        for index, split in enumerate(splits):
            conv = add_conversation(''.join(split))
            response = client.chat.completions.create(messages= conv, model=self.model)
            summary = response.choices[0].message.content
            summaries.append(summary)
            
            if index%4==0:
                print(f'summarized {index+1} out of {len(splits)}') 
        return summaries

    def combine_summaries(self, summaries:list) -> str:
        """Combine list of summaries together.
        
        Parameters
        ----------
        summaries: list of summaries.
        
        """
        summary_str = ' '.join(summaries)  # combine all text from the list into a single string.
        print(f'total number of lines from the individual summary from the list is: {len(summary_str.split('\n'))}')
        summ_conv = add_conversation(summary_str, combined=True)
        response = client.chat.completions.create(messages=summ_conv, model=self.model)
        unified_summy = response.choices[0].message.content
        print(f'total number of lines in the final summary is: {len(unified_summy.split('\n'))}')
        
        return unified_summy

    def summarize_conversation(self, text: str) -> str:
        """Summarize the given conversation.
        
        Parameters
        ----------
        text: str
        
        """
        if self.batch_size:
            splits = create_splits(text, self.batch_size)
        splits = create_splits(text)
        summaries = self.create_summaries(splits)
        summary = self.combine_summaries(summaries)

        return summary


In [118]:
summarizer = ConvoSummarizer(client)
summarizer.summarize_conversation(conv_text)

length of lines in the given conversation is:  2164
summarized 1 out of 11
summarized 5 out of 11
summarized 9 out of 11
total number of lines from the individual summary from the list is: 185
total number of lines in the final summary is: 29


"The conversation revolves around a group of friends discussing various social events, gatherings, and activities in Essen, Germany, and surrounding areas. The main themes and key points can be summarized as follows:\n\n**Social Events and Gatherings:**\n\n1. **Essen Original Event:** The group discusses attending a music festival in Essen, sharing the program and schedule, and deciding on a meeting point.\n2. **Meeting Points:** They choose various meeting points, including Kennedy Platz, Rathaus Galerie, and other locations, to gather before attending events.\n3. **Performances and Concerts:** The group attends and discusses various performances, including the Tom Gregory concert and the Eurovision Song Contest.\n4. **Hiking and Outdoor Activities:** They plan and discuss hiking trips, such as to Altena, and other outdoor activities like cycling and swimming.\n5. **Board Game Nights and Karaoke:** The group organizes and attends board game nights and karaoke events, promoting sociali